In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requests-html

In [50]:
from requests_html import HTMLSession
import pandas as pd 
from bs4 import BeautifulSoup
import re
import unicodedata
from io import StringIO
import time
import numpy as np

In [4]:
session = HTMLSession()
html_file = session.get('https://www.sec.gov/Archives/edgar/full-index/').text
soup = BeautifulSoup(html_file, features="lxml")

In [5]:
for year in range(1993,2022):
    print(year)
   

1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [55]:
#for year in range(1993,2022): (To get a range of all the years)
    #for Q in range(1,5): (To get a range of all the quarters)
for year in range(2019,2020):
    for Q in range(1,2):
        time.sleep(1/10)
        fwf_file = session.get('https://www.sec.gov/Archives/edgar/full-index/' + str(year) + '/' + 'QTR' + str(Q) + '/' + 'crawler.idx').text
        URLs = pd.read_fwf(StringIO(fwf_file), colspecs=[(0,60), (60,72), (72,84), (84,96), (96,200)], skiprows=2)
        URLs.columns=URLs.iloc[0]
        URLs=URLs.iloc[2:]
        URLs = URLs[(URLs['Form Type']=='10-Q')|(URLs['Form Type']=='10-K')]
        #Tested for loop with first 10 rows
        #for i in URLs.index: (To get a range of all the rows in the dataframe)
        for i in URLs.index[0:11]:
            html_file = session.get(URLs['URL'][i]).text
            soup = BeautifulSoup(html_file, features="lxml")
            financial_statement_URL = "https://www.sec.gov"+ soup.table.contents[3].contents[5].a.get('href')
            financials_soup = BeautifulSoup(session.get(financial_statement_URL).text)
            financials = unicodedata.normalize("NFKD", financials_soup.text)
            if i in URLs[(URLs['Form Type']=='10-Q')]['URL']:
                #We tried to extract the third occurance of Item 2 until the third occurance of Item 3 but couldn't get it to run
                MDA_Q_text = re.findall('(?<=Item 2.).*?(?=.Item 3.)', financials, re.DOTALL)
            elif i in URLs[(URLs['Form Type']=='10-K')]['URL']:
                #We tried to extract the third occurance of Item 7 until the third occurance of Item 7A but couldn't get it to run
                MDA_K_text = re.findall('(?<=Item 7.).*?(?=.Item 7A.)', financials, re.DOTALL)
#New dataframe with first 10 rows of data from previous dataframe
MDA_df = URLs[0:11]
#New column in MDA dataframe containing the text where the form type = 10-Q we set it equal to MDA_Q_text otherwise equal to MDA_K_text
#This code was not able to run 
MDA_df['MDA'] = np.where((URLs['Form Type']=='10-Q'),MDA_Q_text, MDA_K_text)

AttributeError: 'list' object has no attribute 'shape'